In [95]:
import requests
import dataretrieval.nwis as nwis
import geopandas as gpd

In [4]:
site="07116500"
meas = nwis.get_record(sites=site, service='measurements')
si = nwis.get_record(sites=site, service='site')

In [5]:
si

,agency_cd,site_no,station_nm,site_tp_cd,lat_va,long_va,dec_lat_va,dec_long_va,coord_meth_cd,coord_acy_cd,...,local_time_fg,reliability_cd,gw_file_cd,nat_aqfr_cd,aqfr_cd,aqfr_type_cd,well_depth_va,hole_depth_va,depth_src_cd,project_no
0,USGS,07116500,"HUERFANO RIVER NEAR BOONE, CO.",ST,381330,1041537,38.225002,-104.2608,M,F,...,Y,NaN,NYNNNNNN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [38]:
for col in si:
    print(f'attribute: {col}, value: {si[col].values[0]}')
#     if col in ['alt_va', 'alt_meth_cd', 'alt_acy_va', 'alt_datum_cd']:
#         print(f'attribute: {col}, value: {si[col].values[0]}')

attribute: agency_cd, value: USGS
attribute: site_no, value: 07116500
attribute: station_nm, value: HUERFANO RIVER NEAR BOONE, CO.
attribute: site_tp_cd, value: ST
attribute: lat_va, value: 381330
attribute: long_va, value: 1041537
attribute: dec_lat_va, value: 38.2250024
attribute: dec_long_va, value: -104.2607996
attribute: coord_meth_cd, value: M
attribute: coord_acy_cd, value: F
attribute: coord_datum_cd, value: NAD27
attribute: dec_coord_datum_cd, value: NAD83
attribute: district_cd, value: 8
attribute: state_cd, value: 8
attribute: county_cd, value: 101
attribute: country_cd, value: US
attribute: land_net_ds, value:   NENES18 T021S R061W
attribute: map_nm, value: nan
attribute: map_scale_fc, value: nan
attribute: alt_va, value: 4443.74
attribute: alt_meth_cd, value: U
attribute: alt_acy_va, value: 15
attribute: alt_datum_cd, value: NGVD29
attribute: huc_cd, value: 11020006
attribute: basin_cd, value: nan
attribute: topo_cd, value: nan
attribute: instruments_cd, value: NNNNYNNNNNN

In [35]:
latstr = 'lat_va'
str = f'N{si[latstr].values[0]}'
str

'N381330'

In [69]:
url = "https://www.ngs.noaa.gov/api/ncat/llh"
lat_str = 'lat_va'
lon_str = 'long_va'
alt_str = 'alt_va'
indatum_str = 'coord_datum_cd'
outdatum_str = 'NAD83(2011)'
inVertDataum_str = 'NGVD29'
outVertDataum = 'NAVD88'

ohgt = float(si[alt_str].values[0])*.3048
repr(ohgt)
# print(lat=f'N{si[latstr].values[0]}'
payload = {
    'lat':f'N{si[latstr].values[0]}',
    'lon':f'W{si[lon_str].values[0]}',
    'orthoHt': repr(ohgt),
    'inDatum':f'{si[indatum_str].values[0]}',
    'outDatum':outdatum_str,
    'inVertDatum':inVertDataum_str,
    'outVertDatum':outVertDataum 
}

In [70]:
payload

{'lat': 'N381330',
 'lon': 'W1041537',
 'orthoHt': '1354.451952',
 'inDatum': 'NAD27',
 'outDatum': 'NAD83(2011)',
 'inVertDatum': 'NGVD29',
 'outVertDatum': 'NAVD88'}

In [80]:
r = requests.get(url, params=payload)
resp = r.json()
print(resp)

{'ID': '1620693787140', 'nadconVersion': '5.0', 'vertconVersion': '3.0', 'srcDatum': 'NAD27', 'destDatum': 'NAD83(2011)', 'srcVertDatum': 'NGVD29', 'destVertDatum': 'NAVD88', 'srcLat': '38.2250000000', 'srcLatDms': 'N381330.00000', 'destLat': '38.2250061349', 'destLatDms': 'N381330.02209', 'deltaLat': '0.681', 'sigLat': '0.002757', 'sigLat_m': '0.0850', 'srcLon': '-104.2602777778', 'srcLonDms': 'W1041537.00000', 'destLon': '-104.2608026762', 'destLonDms': 'W1041538.88963', 'deltaLon': '-45.962', 'sigLon': '0.003251', 'sigLon_m': '0.0791', 'srcEht': 'N/A', 'destEht': 'N/A', 'sigEht': 'N/A', 'srcOrthoht': '1354.452', 'destOrthoht': '1355.236', 'sigOrthoht': '0.002', 'spcZone': 'CO S-0503', 'spcNorthing_m': '478,474.027', 'spcEasting_m': '1,022,903.594', 'spcNorthing_usft': '1,569,793.537', 'spcEasting_usft': '3,355,976.210', 'spcNorthing_ift': '1,569,796.676', 'spcEasting_ift': '3,355,982.921', 'spcConvergence': '00 45 36.35', 'spcScaleFactor': '0.99996861', 'spcCombinedFactor': 'N/A', '

In [77]:
float(resp['destOrthoht'])

1355.236

In [68]:
r

<Response [200]>

In [102]:
def getGageDatum(gagenum, verbose=false):
    si = nwis.get_record(sites=gagenum, service='site')
    if si['alt_datum_cd'].values[0] == 'NGVD29':
#         print('conversion')
        url = "https://www.ngs.noaa.gov/api/ncat/llh"
        lat_str = 'lat_va'
        lon_str = 'long_va'
        alt_str = 'alt_va'
        indatum_str = 'coord_datum_cd'
        outdatum_str = 'NAD83(2011)'
        inVertDataum_str = 'NGVD29'
        outVertDataum = 'NAVD88'
        if f'{si[indatum_str].values[0]}' == 'NAD83':
            indatum = 'NAD83(2011)'
        else:
            indatum = f'{si[indatum_str].values[0]}'

        ohgt = float(si[alt_str].values[0])*.3048
        repr(ohgt)
        # print(lat=f'N{si[latstr].values[0]}'
        payload = {
            'lat':f'N{si[latstr].values[0]}',
            'lon':f'W{si[lon_str].values[0]}',
            'orthoHt': repr(ohgt),
            'inDatum':indatum,
            'outDatum':outdatum_str,
            'inVertDatum':inVertDataum_str,
            'outVertDatum':outVertDataum 
        }
        r = requests.get(url, params=payload)
        resp = r.json()
        if verbose:
            print(f'{si[indatum_str].values[0]}')
            print(resp)
        return float(resp['destOrthoht'])
    else:
#         print('non-conversion')
        return si['alt_va'].values[0]*.3048

NameError: name 'false' is not defined

In [105]:
gages = ['08227500', '06712000', '06762500', '381312104321001', '01011000' ]
for gage in gages:
    print(getGageDatum(gage))

conversion
NAD27
{'ID': '1620758734361', 'nadconVersion': '5.0', 'vertconVersion': '3.0', 'srcDatum': 'NAD27', 'destDatum': 'NAD83(2011)', 'srcVertDatum': 'NGVD29', 'destVertDatum': 'NAVD88', 'srcLat': '38.0136111111', 'srcLatDms': 'N380049.00000', 'destLat': '38.0136163999', 'destLatDms': 'N380049.01904', 'deltaLat': '0.587', 'sigLat': '0.003769', 'sigLat_m': '0.1162', 'srcLon': '-105.6922222222', 'srcLonDms': 'W1054132.00000', 'destLon': '-105.6927946683', 'destLonDms': 'W1054134.06081', 'deltaLon': '-50.270', 'sigLon': '0.012895', 'sigLon_m': '0.3146', 'srcEht': 'N/A', 'destEht': 'N/A', 'sigEht': 'N/A', 'srcOrthoht': '2548.128', 'destOrthoht': '2549.521', 'sigOrthoht': '0.028', 'spcZone': 'CO S-0503', 'spcNorthing_m': '454,308.770', 'spcEasting_m': '897,472.184', 'spcNorthing_usft': '1,490,511.356', 'spcEasting_usft': '2,944,456.657', 'spcNorthing_ift': '1,490,514.337', 'spcEasting_ift': '2,944,462.546', 'spcConvergence': '-00 07 05.72', 'spcScaleFactor': '0.99995029', 'spcCombinedF

KeyError: 'destOrthoht'

In [106]:
site="01011000"
meas = nwis.get_record(sites=site, service='measurements')
si = nwis.get_record(sites=site, service='site')
print(si)

  agency_cd   site_no                           station_nm site_tp_cd  lat_va  \
0      USGS  01011000  Allagash River near Allagash, Maine         ST  470411   

   long_va  dec_lat_va  dec_long_va coord_meth_cd coord_acy_cd  ...  \
0   690446   47.069722   -69.079444             N            S  ...   

  local_time_fg reliability_cd  gw_file_cd  nat_aqfr_cd  aqfr_cd aqfr_type_cd  \
0             Y            NaN    NNNNNNNN          NaN      NaN          NaN   

   well_depth_va hole_depth_va  depth_src_cd  project_no  
0            NaN           NaN           NaN   442300100  

[1 rows x 42 columns]


In [97]:
gages_2_f = '../data/gagesII_9322_point_shapefile/gagesII_9322_sept30_2011.shp'
gages2 = gpd.read_file(gages_2_f)

In [98]:
gages2

,STAID,STANAME,CLASS,AGGECOREGI,DRAIN_SQKM,HUC02,LAT_GAGE,LNG_GAGE,STATE,HCDN_2009,ACTIVE09,FLYRS1900,FLYRS1950,FLYRS1990,geometry
0,01011000,"Allagash River near Allagash, Maine",Non-ref,NorthEast,3186.84400,01,47.069611,-69.079544,ME,None,yes,78.0,60.0,20.0,POINT (2027371.849 2964479.789)
1,01013500,"Fish River near Fort Kent, Maine",Ref,NorthEast,2252.69600,01,47.237394,-68.582642,ME,yes,yes,85.0,60.0,20.0,POINT (2058479.303 2992966.016)
2,01015800,"Aroostook River near Masardis, Maine",Non-ref,NorthEast,2313.75500,01,46.523003,-68.371764,ME,None,yes,51.0,51.0,19.0,POINT (2096533.991 2921785.543)
3,01016500,"MACHIAS RIVER NEAR ASHLAND, ME",Non-ref,NorthEast,847.77210,01,46.628311,-68.434792,ME,None,None,32.0,32.0,0.0,POINT (2088548.849 2931578.094)
4,01017000,"Aroostook River at Washburn, Maine",Non-ref,NorthEast,4278.90700,01,46.777294,-68.157194,ME,None,yes,79.0,60.0,20.0,POINT (2104213.711 2953507.477)
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9317,50141000,"RIO BLANCO NR ADJUNTAS, PR",Ref,PuertoRico,39.33948,21,18.203290,-66.799898,PR,None,None,24.0,21.0,0.0,POINT (3159158.234 -27083.994)
9318,50144000,"RIO GRANDE DE ANASCO NR SAN SEBASTIAN, PR",Non-ref,PuertoRico,347.29680,21,18.282730,-67.051012,PR,None,yes,45.0,45.0,19.0,POINT (3130300.977 -27322.701)
9319,50145395,"RIO CASEI ABV HACIENDA CASEI, PR",Ref,PuertoRico,19.31340,21,18.248287,-67.075179,PR,None,yes,5.0,5.0,5.0,POINT (3128868.889 -31619.830)
9320,50147800,"RIO CULEBRINAS AT HWY 404 NR MOCA, PR",Non-ref,PuertoRico,182.70720,21,18.359670,-67.092124,PR,None,yes,42.0,42.0,20.0,POINT (3123537.324 -20851.868)


In [103]:
for i,row in gages2.iterrows():
    stid = row['STAID']
    print(stid, getGageDatum(stid))

conversion
NAD83
{'error': 'Invalid longitude'}


KeyError: 'destOrthoht'